# BijectiveBase.jl - 对「双射基数n进制」的解析转换支持

## 概述

🎯核心功能：对「双射N进制数值」进行解析、生成

- 与日常所谓「n进制」的区别：**没有表特殊地位的「0」位值**
    - 这意味着「A」与「AA」在任何n下语义都不相同
- 有「数组」「字符串」两种形式可选

## 对照表

部分二进制数的对应关系表如下：

| 原 | BIN | Bijective BIN | 权值显示 |
| ---: | ---: | ---: | ---: |
| 0 | 0 |  |  |
| 1 | 1 | 0 | 1 |
| 2 | 10 | 1 | 2 |
| 3 | 11 | 00 | 21 |
| 4 | 100 | 01 | 22 |
| 5 | 101 | 10 | 41 |
| 6 | 110 | 11 | 42 |
| 7 | 111 | 000 | 421 |
| 8 | 1000 | 001 | 422 |
| 9 | 1001 | 010 | 441 |
| 10 | 1010 | 011 | 442 |
| 11 | 1011 | 100 | 821 |
| 12 | 1100 | 101 | 822 |
| 13 | 1101 | 110 | 841 |
| 14 | 1110 | 111 | 842 |
| 15 | 1111 | 0000 | 8421 |
| $\vdots$ | $\vdots$ | $\vdots$ | $\vdots$ |

^其中**空白单元格**表示「空字符串」

## 使用

该Julia包导出了三个函数，分别为

- `length_bijective`：计算数值在「双射进位制」下的位数
    - `length_bijective(x::Integer, N::Integer) -> Integer`：计算数值$x$在「双射$N$进位制」下的位数
    - `length_bijective(x::Integer, chars::AbstractString) -> Integer`：计算数值$x$在以`chars`为$N$进制字符集的「双射$N$进位制」下的位数
- `num_to_bijective`：将数值转换为双射进位制的符号串
    - `num_to_bijective(x::Integer, N::Integer, f::Function) -> Vector`：将数值$x$通过「符号→位值」的映射$f$转换为双射$N$进位制的符号串
    - `num_to_bijective(x::Integer, chars::AbstractString) -> String`：将数值$x$通过指定的「进制字符集」`chars`转换为双射进位制的字符串
- `bijective_to_num`：将双射进位制的数值转换为数值
    - `bijective_to_num(s::Vector, N::Integer, f⁻¹::Function) -> Integer`：将双射$N$进位制的符号串`s`通过「符号→位值」的逆映射$f^{-1}$转换为数值
    - `bijective_to_num(s::AbstractString, chars::AbstractString) -> Integer`：将双射进位制的符号串`s`通过指定的「进制字符集」`chars`转换为数值

## 参考

- 🔗[双射记数系统 - 维基百科](https://zh.wikipedia.org/wiki/%E9%9B%99%E5%B0%84%E8%A8%98%E6%95%B8)
- 🔗[Bijective numeration - Wikipedia](https://en.wikipedia.org/wiki/Bijective_numeration)

<!-- README-end -->
<!-- TEST-begin -->
## 库代码

<!-- %ignore-cell -->
### 建立模块上下文

In [37]:
#= %only-compiled
module BijectiveBase
%only-compiled =#

### 代码

📌教训：对此类「数值找规律」的问题，一定要善用**🛠️表格对照法**

- ❌闷头写算法：仅凭少量样例编写算法，容易导致过拟合（面对新例出现异常）
- ❌瞎蒙改代码：仅凭一时直觉修改试错，往往思路难拟合（按下葫芦又浮起瓢）

| 原 | BIN | Bijective BIN | $\lceil \log_2 原 \rceil$ | `length(bi-bin)` | 权值显示 |
| ---: | ---: | ---: | ---: | ---: | ---: |
| 0 | 0 |  |  |  |  |
| 1 | 1 | 0 | 1 | 1 | 1 |
| 2 | 10 | 1 | 1 | 1 | 2 |
| 3 | 11 | 00 | 2 | 2 | 21 |
| 4 | 100 | 01 | 2 | 2 | 22 |
| 5 | 101 | 10 | 3 | 2 | 41 |
| 6 | 110 | 11 | 3 | 2 | 42 |
| 7 | 111 | 000 | 3 | 3 | 421 |
| 8 | 1000 | 001 | 3 | 3 | 422 |
| 9 | 1001 | 010 | 4 | 3 | 441 |
| 10 | 1010 | 011 | 4 | 3 | 442 |
| 11 | 1011 | 100 | 4 | 3 | 821 |
| 12 | 1100 | 101 | 4 | 3 | 822 |
| 13 | 1101 | 110 | 4 | 3 | 841 |
| 14 | 1110 | 111 | 4 | 3 | 842 |
| 15 | 1111 | 0000 | 4 | 4 | 8421 |
| $\vdots$ | $\vdots$ | $\vdots$ | $\vdots$ | $\vdots$ | $\vdots$ |

<!-- %ignore-cell -->
暴力算法の铺垫

In [38]:
# %ignore-cell
"生成各个进制下所有不同的位值组合"
iter_desc(N, len) = (
    # 初值
    len == 0 ? [] :
    len == 1 ? ([i] for i in 1:N) :
    # 递归
    (
        [i, desc...]
        for i in 1:N # 外层
        for desc in iter_desc(N, len-1) # 里层
    )
)

"""
    num_to_bijective_BRUTE(x, N, f)
暴力转换算法
- 本质是基于「递归」与「计数原理」
- 🎯保证结果正确，为后续作参照
"""
function num_to_bijective_BRUTE(x, N, f)
    x == 0 && return []
    desc_arr = #= Vector{Int} =#[]
    len = 1
    while length(desc_arr) < x
        push!(desc_arr, iter_desc(N, len)...)
        len += 1
    end
    desc_arr[x] .|> f
end

@show iter_desc(3,4) |> collect
[num_to_bijective_BRUTE(i, 2, x -> x-1) for i in 1:16]

iter_desc(3, 4) |> collect = [[1, 1, 1, 1], [1, 1, 1, 2], [1, 1, 1, 3], [1, 1, 2, 1], [1, 1, 2, 2], [1, 1, 2, 3], [1, 1, 3, 1], [1, 1, 3, 2], [1, 1, 3, 3], [1, 2, 1, 1], [1, 2, 1, 2], [1, 2, 1, 3], [1, 2, 2, 1], [1, 2, 2, 2], [1, 2, 2, 3], [1, 2, 3, 1], [1, 2, 3, 2], [1, 2, 3, 3], [1, 3, 1, 1], [1, 3, 1, 2], [1, 3, 1, 3], [1, 3, 2, 1], [1, 3, 2, 2], [1, 3, 2, 3], [1, 3, 3, 1], [1, 3, 3, 2], [1, 3, 3, 3], [2, 1, 1, 1], [2, 1, 1, 2], [2, 1, 1, 3], [2, 1, 2, 1], [2, 1, 2, 2], [2, 1, 2, 3], [2, 1, 3, 1], [2, 1, 3, 2], [2, 1, 3, 3], [2, 2, 1, 1], [2, 2, 1, 2], [2, 2, 1, 3], [2, 2, 2, 1], [2, 2, 2, 2], [2, 2, 2, 3], [2, 2, 3, 1], [2, 2, 3, 2], [2, 2, 3, 3], [2, 3, 1, 1], [2, 3, 1, 2], [2, 3, 1, 3], [2, 3, 2, 1], [2, 3, 2, 2], [2, 3, 2, 3], [2, 3, 3, 1], [2, 3, 3, 2], [2, 3, 3, 3], [3, 1, 1, 1], [3, 1, 1, 2], [3, 1, 1, 3], [3, 1, 2, 1], [3, 1, 2, 2], [3, 1, 2, 3], [3, 1, 3, 1], [3, 1, 3, 2], [3, 1, 3, 3], [3, 2, 1, 1], [3, 2, 1, 2], [3, 2, 1, 3], [3, 2, 2, 1], [3, 2, 2, 2], [3, 2, 2, 3], [3, 

16-element Vector{Vector{Int64}}:
 [0]
 [1]
 [0, 0]
 [0, 1]
 [1, 0]
 [1, 1]
 [0, 0, 0]
 [0, 0, 1]
 [0, 1, 0]
 [0, 1, 1]
 [1, 0, 0]
 [1, 0, 1]
 [1, 1, 0]
 [1, 1, 1]
 [0, 0, 0, 0]
 [0, 0, 0, 1]

计算长度

In [39]:
# ! Jupyter允许在单元格中导出符号（而无视模块上下文）
export length_bijective

"""
    length_bijective(x::I, N) -> Integer where {I <: Integer}
计算「双射进位制数」的长度（基数版）
- @param x 需要转换的数值
- @param N 进制基数
- @returns 所转换成的「双射N进位数」的基数
"""
function length_bijective(x::I, N::U) where {I <: Integer, U <: Integer}
    local n::I = 0
    local y::I = x
    while y >= N^n
        y -= N^n
        n += 1
    end
    return n
end
"""
    length_bijective(x, chars::AbstractString) -> Integer
计算「双射进位制数」的长度（字符串版）
- @param x 需要转换的数值
- @param chars 进制字符集
- @returns 🔗以「字符集大小」为进制基数
"""
length_bijective(x, chars::AbstractString) = length_bijective(x, length(chars))
# %ignore-below # * 测试代码: 验证长度正确

@assert length_bijective.(0:15, 1) == collect(0:15) # 一进制就是堆叠
@assert length_bijective.(0:15, 2) == [0, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4] # 二进制
@assert length_bijective.(0:15, 3) == [0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3] # 三进制
let test(N, range) = @assert all(length_bijective.(range, N) .== length.(num_to_bijective_BRUTE.(range, N, identity)))
    for N in 1:10
        range_max = 20 ÷ N
        test(N, 0:range_max)
        @info "长度测试成功！" N range_max
    end
end

┌ Info: 长度测试成功！
│   N = 1
└   range_max = 20
┌ Info: 长度测试成功！
│   N = 2
└   range_max = 10
┌ Info: 长度测试成功！
│   N = 3
└   range_max = 6
┌ Info: 长度测试成功！
│   N = 4
└   range_max = 5
┌ Info: 长度测试成功！
│   N = 5
└   range_max = 4
┌ Info: 长度测试成功！
│   N = 6
└   range_max = 3
┌ Info: 长度测试成功！
│   N = 7
└   range_max = 2
┌ Info: 长度测试成功！
│   N = 8
└   range_max = 2
┌ Info: 长度测试成功！
│   N = 9
└   range_max = 2
┌ Info: 长度测试成功！
│   N = 10
└   range_max = 2


数组版本

In [40]:
# ! Jupyter允许在单元格中导出符号（而无视模块上下文）
export num_to_bijective, bijective_to_num

"""
    num_to_bijective(x::I, N::U, f::Function, T::Type=Any) -> Vector{T} where {I <: Integer, U <: Integer}
原数→双射进位制数（数组版本）
- ⚠️其中返回的数组对「索引」而言是「从高到底数」的
    - 遵循字面呈现规则，如「双射三进制」下`101`被直译为`[1, 0, 1]`
    - 📌若后续需要扩展，可能需要倒序
"""
function num_to_bijective(x::I, N::U, f::Function, T::Type=Any) where {I <: Integer, U <: Integer}
    # ! 通用，无需考虑x=0的情况

    # 减去1111，并得到长度 | 将「1~N」问题 转换为 「0~(N-1)」问题
    local n::I = 0
    local y::I = x
    while y >= N^n
        y -= N^n
        n += 1
    end

    # 将y按照常规的「除N取余」来做 | 已转换为「0~(N-1)」问题
    local s::Vector{T} = Vector{T}(undef, n)
    local c::I
    while n > 0
        y, c = divrem(y, N) # 除N取余
        s[n] = f(c + 1) # 计入
        n -= 1 # 自减
    end

    # 返回最终结果
    return s
end
     
"""
    bijective_to_num(s::Vector{T}, N::U, f⁻¹::Function) -> Integer
双射进制数→原数（数组版本）
"""
function bijective_to_num(s::Vector{T}, N::U, f⁻¹::Function) where {T, U <: Integer}
    isempty(s) && return 0
    local l::Integer = length(s)
    return sum(
        f⁻¹(s[l-i]) * N^i
        for i in 0:(l-1)
    )
end
# %ignore-below # * 测试代码

# 尝试使用数据框
has_DF = try
    using DataFrames: DataFrame
    true
catch
    @warn "DataFrames包未启用！"
    false
end
# 正式开始
let test(N = 2, NUM = 16) = begin
    f(x) = x#-1
    f⁻¹(x) = x#+1
    parseInt(x) = isempty(x) ? 0 : parse(Int, x)

    # 验证空值
    @assert isempty(num_to_bijective(0, N, f))
    @assert isempty(num_to_bijective_BRUTE(0, N, f))

    # 穷举数据 | 使用广播方法
    range = 1:NUM
    num = N <= 1 ? [nothing for _ in range] : string.(range; base=N)
    len = length_bijective.(range, N)
    arr = num_to_bijective.(range, N, f) .|> Vector{Int}
    arr_B = num_to_bijective_BRUTE.(range, N, f) .|> Vector{Int}
    arr_r = bijective_to_num.(arr, N, f⁻¹)
    # 可选地启用DF进行展示
    df = has_DF ? DataFrame(
        # :i => range,
        :num => num,
        :len => len,
        :arr_B => arr_B,
        :arr => arr,
        :arr_r => arr_r,
        :eq => arr .== arr_B,
    ) : nothing
    @assert all(arr .== arr_B) # 与正确的「暴力算法」结果相同
    @assert all(arr_r .== range) # 正逆向转换不损失信息
    # df[df[!, :eq] .⊻ true, :]
    has_DF ? df : arr
end
test.([1, 2, 3, 4])
end

4-element Vector{DataFrame}:
 16×6 DataFrame
 Row │ num      len    arr_B                              arr                  ⋯
     │ Nothing  Int64  Array…                             Array…               ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │              1  [1]                                [1]                  ⋯
   2 │              2  [1, 1]                             [1, 1]
   3 │              3  [1, 1, 1]                          [1, 1, 1]
   4 │              4  [1, 1, 1, 1]                       [1, 1, 1, 1]
   5 │              5  [1, 1, 1, 1, 1]                    [1, 1, 1, 1, 1]      ⋯
   6 │              6  [1, 1, 1, 1, 1, 1]                 [1, 1, 1, 1, 1, 1]
   7 │              7  [1, 1, 1, 1, 1, 1, 1]              [1, 1, 1, 1, 1, 1, 1
   8 │              8  [1, 1, 1, 1, 1, 1, 1, 1]           [1, 1, 1, 1, 1, 1, 1
   9 │              9  [1, 1, 1, 1, 1, 1, 1, 1, 1]        [1, 1, 1, 1, 1, 1, 1 ⋯
  10 │             10  [1, 1, 

字符串版本

In [41]:
# * 一些工具函数

"【内部】获取指定*位置*的字符（无视Unicode多字节限制）"
@inline char_at(s::AbstractString, position::Integer) = s[nextind(s, 0, position)]

"【内部】根据字符获取首次出现的*位置*"
function first_index(s::AbstractString, c::AbstractChar)
    local i = firstindex(s)
    local position = 1
    while i <= lastindex(s)
        # 相等⇒返回
        s[i] === c && return position
        # 否则⇒递增
        i = nextind(s, i)
        position += 1
    end
    return nothing
end
"反过来也行"
first_index(c::AbstractChar, s::AbstractString) = first_index(s, c)
# %ignore-begin # *测试 
let s = "一二三四五六七八九十"
    test_set = [
        3 => '三'
        5 => '五'
        9 => '九'
    ]
    for (position, c) in test_set
        @assert char_at(s, position) === c
        @assert first_index(c, s) == position
    end
end
# %ignore-end

# ! 函数已导出，此处只是添加了不同的方法

"""
    num_to_bijective(x::I, chars::AbstractString) where {I <: Integer} -> Integer
原数→双射进位制数（字符串版本）
- 自动以「字符集大小」作为基数
- ⚠️其中返回的数组对「索引」而言是「从高到底数」的
    - 遵循字面呈现规则，如「双射三进制」下`101`即字符串"101"
    - 📌若后续需要扩展，可能需要倒序读取
"""
function num_to_bijective(x::I, chars::AbstractString) where {I <: Integer}
    # ! 通用，无需考虑x=0的情况

    # 通过字串长度获得基数N
    local N = length(chars)

    # 减去1111，并得到长度 | 将「1~N」问题 转换为 「0~(N-1)」问题
    local n::I = 0
    local y::I = x
    while y >= N^n
        y -= N^n
        n += 1
    end

    # 将y按照常规的「除N取余」来做 | 已转换为「0~(N-1)」问题
    local s::Vector{Char} = Vector{Char}(undef, n)
    local c::I
    while n > 0
        y, c = divrem(y, N) # 除N取余
        s[n] = char_at(chars, c+1) # 计入
        n -= 1 # 自减
    end

    # 返回最终结果
    return join(s)
end
     
"""
    bijective_to_num(s::AbstractString, chars::AbstractString)
双射进制数→原数（字符串版本）
- 自动以「字符集大小」作为基数
"""
function bijective_to_num(s::AbstractString, chars::AbstractString)
    isempty(s) && return 0
    local N = length(chars)
    local l = length(s)
    return sum(
        first_index(chars, char_at(s, l-i)) * N^i
        for i in 0:(l-1)
    )
end
# %ignore-below # * 测试代码

# 尝试使用数据框
has_DF = try
    using DataFrames: DataFrame
    true
catch
    @warn "DataFrames包未启用！"
    false
end
# 正式开始
let test(chars::AbstractString, NUM = 16) = begin
    N = length(chars)
    parseInt(x) = isempty(x) ? 0 : parse(Int, x)

    # 验证空值
    @assert isempty(num_to_bijective(0, chars))
    # @assert isempty(num_to_bijective_BRUTE(0, N, chars))

    # 穷举数据 | 使用广播方法 # ! 此处不再与暴力算法作对比
    range = 1:NUM
    num = N <= 1 ? [nothing for _ in range] : string.(range; base=N)
    len = length_bijective.(range, chars)
    arr = num_to_bijective.(range, chars)
    arr_r = bijective_to_num.(arr, chars)
    eq = arr_r .== range
    # 可选地启用DF进行展示
    df = has_DF ? DataFrame(
        # :i => range,
        :num => num,
        :len => len,
        :arr => arr,
        :arr_r => arr_r, # ! ↓全部相等就不展示了
        (all(eq) ? [] : [:eq => eq])...,
    ) : nothing
    @assert all(eq) # 正逆向转换不损失信息
    # df[df[!, :eq] .⊻ true, :]
    has_DF ? df : arr
end
test.(["1", "12", "123", "1234"])
test.(["一", "一二", "一二三", "一二三四"])
end

4-element Vector{DataFrame}:
 16×4 DataFrame
 Row │ num      len    arr                               arr_r 
     │ Nothing  Int64  String                            Int64 
─────┼─────────────────────────────────────────────────────────
   1 │              1  一                                    1
   2 │              2  一一                                  2
   3 │              3  一一一                                3
   4 │              4  一一一一                              4
   5 │              5  一一一一一                            5
   6 │              6  一一一一一一                          6
   7 │              7  一一一一一一一                        7
   8 │              8  一一一一一一一一                      8
   9 │              9  一一一一一一一一一                    9
  10 │             10  一一一一一一一一一一                 10
  11 │             11  一一一一一一一一一一一               11
  12 │             12  一一一一一一一一一一一一             12
  13 │             13  一一一一一一一一一一一一一           13
  14 │             14  一一一一一一一一一一一一

<!-- %ignore-cell -->
### 终止模块上下文

In [42]:
#= %only-compiled
end # module
%only-compiled =#

<!-- %ignore-cell -->
<!-- TEST-end -->
## 构建代码

<!-- %ignore-cell -->
✨**Powered by [IpynbCompile.jl](https://github.com/ARCJ137442/IpynbCompile.jl)**

<!-- %ignore-cell -->
### 预先确定变量

<!-- %ignore-cell -->
路径变量

In [43]:
# %ignore-cell

# 列举文件名
SELF_FILE = "BijectiveBase.ipynb"
SELF_JL = "BijectiveBase.jl"
README_FILE = "README.md"
TEST_JL = "runtests.jl"

# 计算根路径
ROOT_PATH = (
    contains(@__DIR__, "src")
        ? dirname(@__DIR__)
        : @__DIR__
)

# 计算各文件路径
SELF_PATH = joinpath(ROOT_PATH, "src", SELF_FILE)
SELF_PATH_JL = joinpath(ROOT_PATH, "src", SELF_JL)
README_PATH = joinpath(ROOT_PATH, README_FILE)
TEST_PATH = joinpath(ROOT_PATH, "test", TEST_JL)

# 验证路径存在
@assert ispath(SELF_PATH)
@assert ispath(SELF_PATH_JL)
@assert ispath(README_PATH)
@assert ispath(TEST_PATH)

<!-- %ignore-cell -->
导入外部库

In [44]:
# %ignore-cell
using IpynbCompile

<!-- %ignore-cell -->
自身笔记本

In [45]:
# %ignore-cell
SELF_NOTEBOOK = read_notebook(SELF_PATH)

IpynbNotebook{IpynbCell}(IpynbCell[IpynbCell("markdown", ["# BijectiveBase.jl - 对「双射基数n进制」的解析转换支持"], Dict{String, Any}(), nothing), IpynbCell("markdown", ["## 概述"], Dict{String, Any}(), nothing), IpynbCell("markdown", ["🎯核心功能：对「双射N进制数值」进行解析、生成\n", "\n", "- 与日常所谓「n进制」的区别：**没有表特殊地位的「0」位值**\n", "    - 这意味着「A」与「AA」在任何n下语义都不相同\n", "- 有「数组」「字符串」两种形式可选"], Dict{String, Any}(), nothing), IpynbCell("markdown", ["## 对照表"], Dict{String, Any}(), nothing), IpynbCell("markdown", ["部分二进制数的对应关系表如下：\n", "\n", "| 原 | BIN | Bijective BIN | 权值显示 |\n", "| ---: | ---: | ---: | ---: |\n", "| 0 | 0 |  |  |\n", "| 1 | 1 | 0 | 1 |\n", "| 2 | 10 | 1 | 2 |\n", "| 3 | 11 | 00 | 21 |\n", "| 4 | 100 | 01 | 22 |\n", "| 5 | 101 | 10 | 41 |\n"  …  "| 9 | 1001 | 010 | 441 |\n", "| 10 | 1010 | 011 | 442 |\n", "| 11 | 1011 | 100 | 821 |\n", "| 12 | 1100 | 101 | 822 |\n", "| 13 | 1101 | 110 | 841 |\n", "| 14 | 1110 | 111 | 842 |\n", "| 15 | 1111 | 0000 | 8421 |\n", "| \$\\vdots\$ | \$\\vdots\$ | \$\\vdots\$ | \$\\vdots\$ |\

<!-- %ignore-cell -->
### 自编译生成`BijectiveBase.jl`

In [46]:
# %ignore-cell # * 直接将自身笔记本编译到JL路径
compile_notebook(SELF_NOTEBOOK, SELF_PATH_JL)

8944

<!-- %ignore-cell -->
### 编译生成测试文件`runtests.jl`

🔗Copy from `IpynbCompile.ipynb`

In [47]:
# %ignore-cell
let 测试の始の标签 = "<!-- TEST-begin",
    测试の终の标签 = "<!-- TEST-end"
    
    # 获取测试代码的框定范围 # ! 不包括加了标签的两个单元格
    测试の索引(标签, findF, default=nothing) = something(
        findF(SELF_NOTEBOOK.cells) do cell
            # * 单元格内任意一行以标签开始，就算
            any(startswith(line, 标签) for line in cell.source)
        end,
        default
    )
    测试の始の索引 = 测试の索引(测试の始の标签, findfirst, firstindex(SELF_NOTEBOOK.cells))
    测试の终の索引 = 测试の索引(测试の终の标签, findlast, lastindex(SELF_NOTEBOOK.cells))
    测试の单元格の组 = SELF_NOTEBOOK.cells[(测试の始の索引+1):(测试の终の索引-1)]

    # 拼接所有代码单元格
    code_tests = join((
        join(cell.source)
        for cell in 测试の单元格の组
        if cell.cell_type == "code"
    ), "\n\n")
    # 开头使用Test库，并添加测试上下文
    code_tests = """\
    # 【附加】使用测试代码
    using Test

    @testset "main" begin
    """ * code_tests
    # 替换所有的`@assert`为`@test`
    code_tests = replace(code_tests, "@assert" => "@test")
    # 避免 syntax: malformed "export" statement
    code_tests = replace(code_tests, r"[ \t\v]*export [^\n]*" => "\n#= export导出已忽略 =#")
    # 注释掉所有的`write`写入代码（单行）
    code_tests = replace(
        code_tests, 
        # * 📝Julia中的「捕获-映射」替换：传入一个函数✅
        r"\n *write\(([^\n]+)\)(?:\n|$)" => "\n#= 文件读写已忽略 =#\n"
    )
    # 关闭测试上下文
    code_tests *= """
    
    end
    """
    # 最终写入
    write_bytes = write(TEST_PATH, code_tests)
    printstyled(
        "✅测试文件编译成功！\n（共写入 $write_bytes 个字节）\n";
        color=:light_green, bold=true
    )
end

✅测试文件编译成功！
（共写入 8809 个字节）


<!-- %ignore-cell -->
### 编译生成自述文件`README.md`

🔗Copy from `IpynbCompile.ipynb`

In [48]:
# %ignore-cell # * 扫描自身Markdown单元格，自动生成`README.md`
"决定「单元格采集结束」的标识"
FLAG_END = "<!-- README-end"
FLAG_IGNORE = "<!-- %ignore-cell" # !【2024-01-30 15:03:43】现在统一语法

# * 过滤Markdown单元格
markdowns = filter(SELF_NOTEBOOK.cells) do cell
    cell.cell_type == "markdown"
end
# * 截取Markdown单元格 | 直到开头有`FLAG_END`标记的行（不考虑换行符）
README_END_INDEX = findlast(markdowns) do cell
    !isempty(cell.source) && startswith(cell.source[begin], FLAG_END)
end
README_markdowns = markdowns[begin:README_END_INDEX-1]

# * 提取Markdown代码，聚合生成原始文档
README_markdown_TEXT = join((
    join(cell.source) * '\n' # ←这里需要加上换行
    for cell in README_markdowns
    # 根据【空单元格】或【首行注释】进行忽略
    if !(isempty(cell.source) || startswith(cell.source[begin], FLAG_IGNORE))
), '\n')

# * 继续处理：缩进4→2，附加注释
README_markdown_TEXT = join((
    begin
        local space_stripped_line = lstrip(line, ' ')
        local head_space_length = length(line) - length(space_stripped_line)
        # 缩进缩减到原先的一半
        ' '^(head_space_length ÷ 2) * space_stripped_line
    end
    for line in split(README_markdown_TEXT, '\n')
), '\n')
using Dates: now # * 增加日期注释（不会在正文显示）
README_markdown_TEXT = """\
<!-- ⚠️该文件由 `$SELF_FILE` 自动生成于 $(now())，无需手动修改 -->
$README_markdown_TEXT\
"""
print(README_markdown_TEXT)

README_FILE = "README.md"
write(joinpath(ROOT_PATH, README_FILE), README_markdown_TEXT)

<!-- ⚠️该文件由 `BijectiveBase.ipynb` 自动生成于 2024-01-31T15:34:08.485，无需手动修改 -->
# BijectiveBase.jl - 对「双射基数n进制」的解析转换支持

## 概述

🎯核心功能：对「双射N进制数值」进行解析、生成

- 与日常所谓「n进制」的区别：**没有表特殊地位的「0」位值**
  - 这意味着「A」与「AA」在任何n下语义都不相同
- 有「数组」「字符串」两种形式可选

## 对照表

部分二进制数的对应关系表如下：

| 原 | BIN | Bijective BIN | 权值显示 |
| ---: | ---: | ---: | ---: |
| 0 | 0 |  |  |
| 1 | 1 | 0 | 1 |
| 2 | 10 | 1 | 2 |
| 3 | 11 | 00 | 21 |
| 4 | 100 | 01 | 22 |
| 5 | 101 | 10 | 41 |
| 6 | 110 | 11 | 42 |
| 7 | 111 | 000 | 421 |
| 8 | 1000 | 001 | 422 |
| 9 | 1001 | 010 | 441 |
| 10 | 1010 | 011 | 442 |
| 11 | 1011 | 100 | 821 |
| 12 | 1100 | 101 | 822 |
| 13 | 1101 | 110 | 841 |
| 14 | 1110 | 111 | 842 |
| 15 | 1111 | 0000 | 8421 |
| $\vdots$ | $\vdots$ | $\vdots$ | $\vdots$ |

^其中**空白单元格**表示「空字符串」

## 使用

该Julia包导出了三个函数，分别为

- `length_bijective`：计算数值在「双射进位制」下的位数
  - `length_bijective(x::Integer, N::Integer) -> Integer`：计算数值$x$在「双射$N$进位制」下的位数
  - `length_bijective(x::Integer, chars::AbstractString) -> Integer`：计算数值$x$在以`chars`为$N$进制字符集

2498